Generative Adversarial Networks (GANs)

GANs are a type of neural network that generate new data similar to a given dataset. 

They are widely used for image generation, deepfake creation, and style transfer.

What is a GAN?

A GAN consists of two networks:

Generator  – Creates fake data (e.g., fake images).

Discriminator  – Tries to distinguish between real and fake data.

 The Generator tries to fool the Discriminator.

 The Discriminator gets better at spotting fakes, forcing the Generator to improve.

GAN Architecture

Generator takes random noise and creates fake images.

Discriminator classifies images as real or fake.

Both networks train in a min-max game:

Generator tries to minimize detection by the discriminator.

Discriminator tries to maximize accuracy.

 Loss Function:

The goal of GANs is to minimize the Discriminator's ability to distinguish real from fake images.

Implementing a Simple GAN in TensorFlow (Step-by-Step)

Step 1: Import Libraries

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt


Step 2:

 Build the Generator Network

The Generator takes random noise and creates a 28×28 grayscale image (like MNIST digits).

In [ ]:
def build_generator():
    model = keras.Sequential([
        layers.Dense(128, activation="relu", input_shape=(100,)),  # Input: Noise vector (100)
        layers.BatchNormalization(),
        layers.Dense(256, activation="relu"),
        layers.BatchNormalization(),
        layers.Dense(28 * 28, activation="sigmoid"),  # Output: 28x28 pixels
        layers.Reshape((28, 28))
    ])
    return model

generator = build_generator()
generator.summary()


Step 3:

 Build the Discriminator Network

The Discriminator takes an image (real or fake) and classifies it as real (1) or fake (0).



In [ ]:
def build_discriminator():
    model = keras.Sequential([
        layers.Flatten(input_shape=(28, 28)),
        layers.Dense(256, activation="relu"),
        layers.Dense(128, activation="relu"),
        layers.Dense(1, activation="sigmoid")  # Output: Probability (real or fake)
    ])
    return model

discriminator = build_discriminator()
discriminator.summary()


Step 4:

 Compile the Discriminator

We compile the discriminator using binary crossentropy loss.

In [ ]:
discriminator.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])


Step 5:

 Create and Compile the GAN Model

Freeze the Discriminator (so it doesn’t learn too fast).

Train only the Generator to fool the Discriminator.

In [ ]:
discriminator.trainable = False

# Combine Generator and Discriminator
gan_input = keras.Input(shape=(100,))
fake_image = generator(gan_input)
gan_output = discriminator(fake_image)

gan = keras.Model(gan_input, gan_output)
gan.compile(optimizer="adam", loss="binary_crossentropy")


Step 6:

 Train the GAN

Train Discriminator on real & fake images.

Train Generator to fool the Discriminator

In [ ]:
(x_train, _), _ = keras.datasets.mnist.load_data()
x_train = (x_train / 255.0).reshape(-1, 28, 28)  # Normalize to [0,1]

batch_size = 128
epochs = 10000

for epoch in range(epochs):
    # Train Discriminator
    idx = np.random.randint(0, x_train.shape[0], batch_size)
    real_images = x_train[idx]
    
    noise = np.random.normal(0, 1, (batch_size, 100))
    fake_images = generator.predict(noise)
    
    d_loss_real = discriminator.train_on_batch(real_images, np.ones((batch_size, 1)))
    d_loss_fake = discriminator.train_on_batch(fake_images, np.zeros((batch_size, 1)))
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

    # Train Generator
    noise = np.random.normal(0, 1, (batch_size, 100))
    g_loss = gan.train_on_batch(noise, np.ones((batch_size, 1)))  # Want to trick Discriminator

    # Print progress
    if epoch % 1000 == 0:
        print(f"Epoch {epoch}: D Loss = {d_loss[0]}, G Loss = {g_loss}")

print("Training complete!")


Generate New Images with the Trained GAN


In [ ]:
# Generate new images
noise = np.random.normal(0, 1, (5, 100))  # 5 random noise vectors
generated_images = generator.predict(noise)

# Display images
fig, axs = plt.subplots(1, 5, figsize=(10, 2))
for i in range(5):
    axs[i].imshow(generated_images[i], cmap='gray')
    axs[i].axis('off')
plt.show()


Advanced GAN Variants

**GAN Type**	**Feature** 	**Use Case**

**DCGAN (Deep Convolutional GAN)**	Uses CNNs instead of dense layers	Image generation

**SRGAN (Super-Resolution GAN)**	Enhances image resolution	Image upscaling

**CycleGAN**	Transforms images between domains	Style transfer (e.g., horses → zebras)

**Pix2Pix**	Transforms sketches into images	Image-to-image translation

Real-World Applications of GANs


Deepfakes  – Face swapping in videos.

Art Generation – AI-generated paintings.

Super-Resolution – Enhancing low-res images.

Data Augmentation – Creating synthetic training data.